# Clustering polls into topics
> Using titles/descriptions of polls for clustering. Goal is unsupervised grouping into topicsish based on [this](https://radimrehurek.com/gensim/auto_examples/core/run_topics_and_transformations.html#sphx-glr-auto-examples-core-run-topics-and-transformations-py).

> Note: You may need to run `python -m spacy download de_core_news_sm`, if not already done, to process the German language.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pprint

import pandas as pd

import bundestag.logging as logging
import bundestag.paths as paths
import bundestag.poll_clustering as pc

logger = logging.logger
logger.setLevel("DEBUG")

_paths = paths.get_paths("../data")
_paths

In [ ]:
file = _paths.preprocessed_abgeordnetenwatch / "polls_111.parquet"
file

In [ ]:
df_polls = pd.read_parquet(file)
df_polls.head(3).T

## Clustering based on poll title



Sanity checking word counts, longest and shortest titles

### Cleaning using spacy

https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0

In [ ]:
# !python -m spacy download de_core_news_sm

In [ ]:
col = "poll_title"
nlp_col = f"{col}_nlp_processed"

In [ ]:
st = pc.SpacyTransformer()
df_polls[nlp_col] = df_polls.pipe(pc.clean_text, col=col, nlp=st.nlp)

In [ ]:
df_polls.head(3).T

### Inspecting word frequencies

In [ ]:
pc.compare_word_frequencies(df_polls, col, nlp_col)

The word count distribution shifted to lower values, as could be expected, but no documents were left without any words.

### Transforming using LDA

In [ ]:
st.fit(df_polls[nlp_col].values, mode="lda", num_topics=10)
print("Discovered topics:")
pprint.pprint(st.lda_topics)

In [ ]:
df_lda = st.transform_documents(df_polls[nlp_col])
df_lda.head().T

In [ ]:
df_polls, nlp_feature_cols = df_polls.pipe(
    st.transform, col=nlp_col, return_new_cols=True
)
df_polls.head()

In [ ]:
pc.pca_plot_lda_topics(df_polls, st, col, nlp_feature_cols)